In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('s3://creditdata2080/Datos/Original/german_credit_data.csv')

In [10]:
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


## Exploracion

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


In [12]:
df.describe()

,Unnamed: 0,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,35.546000,1.904000,3271.258000,20.903000
std,288.819436,11.375469,0.653614,2822.736876,12.058814
min,0.000000,19.000000,0.000000,250.000000,4.000000
25%,249.750000,27.000000,2.000000,1365.500000,12.000000
50%,499.500000,33.000000,2.000000,2319.500000,18.000000
75%,749.250000,42.000000,2.000000,3972.250000,24.000000
max,999.000000,75.000000,3.000000,18424.000000,72.000000


In [13]:
df['Risk'].value_counts()

good    700
bad     300
Name: Risk, dtype: int64

In [14]:
train, test = train_test_split(df, test_size = 0.3, random_state = 42,stratify = df['Risk'])

In [15]:
train['Risk'].value_counts()/train['Risk'].count()

good    0.7
bad     0.3
Name: Risk, dtype: float64

In [16]:
test['Risk'].value_counts()/test['Risk'].count()

good    0.7
bad     0.3
Name: Risk, dtype: float64

In [17]:
train.to_csv('s3://creditdata2080/Datos/train/train.csv', index = False)

In [18]:
test.to_csv('s3://creditdata2080/Datos/test/train.csv',index = False)

In [19]:
#Verificar la cantidad de valores nulos en el dataset
train.isna().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     125
Checking account    274
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [20]:
#Cantidad de reuniones por grupo
train.groupby(['Risk']).size().reset_index(name='cantidad de reuniones por grupo')

,Risk,cantidad de reuniones por grupo
0,bad,210
1,good,490


In [21]:
#Cantidad de reuniones por grupo
train.groupby(['Risk','Sex']).size().reset_index(name='cantidad de reuniones por grupo')

,Risk,Sex,cantidad de reuniones por grupo
0,bad,female,76
1,bad,male,134
2,good,female,138
3,good,male,352


In [22]:
#Cantidad de reuniones por grupo
train.groupby(['Sex']).size().reset_index(name='cantidad de reuniones por grupo')

,Sex,cantidad de reuniones por grupo
0,female,214
1,male,486


In [23]:
#Cantidad de reuniones por grupo
train.groupby(['Age','Risk']).size().reset_index(name='cantidad de reuniones por grupo')

,Age,Risk,cantidad de reuniones por grupo
0,20,bad,3
1,20,good,8
2,21,bad,3
3,21,good,5
4,22,bad,9
...,...,...,...
89,68,bad,2
90,68,good,1
91,70,good,1
92,74,good,1


In [24]:
pd.crosstab(index=train['Housing'], columns=train['Risk'], margins =True)

Risk,bad,good,All
Housing,,,
free,29,45,74
own,133,371,504
rent,48,74,122
All,210,490,700


In [25]:
tabla_sex=pd.crosstab(index=train['Sex'], columns=train['Risk'], margins =True)

In [26]:
tabla_sex

Risk,bad,good,All
Sex,,,
female,76,138,214
male,134,352,486
All,210,490,700


In [27]:
tabla_sex['probabilidad'] = tabla_sex.iloc[:,1]/tabla_sex.iloc[:,2]

In [28]:
tabla_sex

Risk,bad,good,All,probabilidad
Sex,,,,
female,76,138,214,0.64486
male,134,352,486,0.72428
All,210,490,700,0.70000


## Preprocesamiento

## Imputación

In [29]:
from sklearn.impute import SimpleImputer
import numpy as np

In [66]:
X_train = train[['Age', 'Credit amount','Duration']]

In [52]:
imputer = SimpleImputer(strategy = 'mean')

In [32]:
imputer.statistics_

AttributeError: 'SimpleImputer' object has no attribute 'statistics_'

In [53]:
X_train['Age'].mean()

35.402857142857144

In [54]:
imputer.transform([[24,100,np.nan], [np.nan,np.nan,12]])

NotFittedError: This SimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [55]:
imputer.fit_transform(X_train)

array([[3.100e+01, 4.473e+03, 3.600e+01],
       [4.600e+01, 1.829e+03, 1.500e+01],
       [2.700e+01, 7.418e+03, 6.000e+01],
       ...,
       [6.300e+01, 1.655e+03, 1.200e+01],
       [4.900e+01, 2.096e+03, 1.200e+01],
       [3.700e+01, 3.676e+03, 6.000e+00]])

In [56]:
X_train = train[['Saving accounts', 'Checking account']]

In [38]:
imputer = SimpleImputer(strategy = 'most_frequent')

In [39]:
imputer.fit(X_train)

SimpleImputer(strategy='most_frequent')

In [40]:
imputer.statistics_

array(['little', 'little'], dtype=object)

In [41]:
imputer.fit_transform(X_train)

array([['little', 'rich'],
       ['little', 'little'],
       ['little', 'moderate'],
       ...,
       ['little', 'little'],
       ['little', 'little'],
       ['little', 'little']], dtype=object)

In [57]:
from sklearn.impute import KNNImputer

In [43]:
imputer = KNNImputer(n_neighbors = 2)

In [44]:
df2 = pd.DataFrame([[40, 19, 3], [42,20,3.1], [44,21,np.nan], [45,23,4.1], [39,25, 5.0], [80,27,np.nan], [82,3.0,4.8]], columns=["Peso", "Edad", "nota"])

In [45]:
df2

,Peso,Edad,nota
0,40,19.0,3.0
1,42,20.0,3.1
2,44,21.0,NaN
3,45,23.0,4.1
4,39,25.0,5.0
5,80,27.0,NaN
6,82,3.0,4.8


In [46]:
result = imputer.fit_transform(df2)

In [47]:
result

array([[40.  , 19.  ,  3.  ],
       [42.  , 20.  ,  3.1 ],
       [44.  , 21.  ,  3.6 ],
       [45.  , 23.  ,  4.1 ],
       [39.  , 25.  ,  5.  ],
       [80.  , 27.  ,  4.45],
       [82.  ,  3.  ,  4.8 ]])

## Escalada

In [90]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [60]:
scaler = StandardScaler()

In [64]:
X_train

,Saving accounts,Checking account
328,little,rich
891,little,NaN
255,NaN,moderate
243,little,NaN
492,moderate,NaN
...,...,...
73,little,moderate
401,NaN,moderate
769,little,NaN
2,little,NaN


In [67]:
scaler.fit(X_train[['Age']])

StandardScaler()

In [68]:
scaler.mean_

array([35.40285714])

In [69]:
scaler.var_**0.5

array([11.23479253])

In [70]:
scaler.transform(X_train[['Age']])

array([[-0.39189483],
       [ 0.94324331],
       [-0.74793167],
       [-0.74793167],
       [-0.74793167],
       [ 0.05315121],
       [-0.035858  ],
       [-1.0149593 ],
       [-0.48090404],
       [ 1.12126173],
       [-0.48090404],
       [-1.28198693],
       [ 1.65531698],
       [-1.10396851],
       [-0.65892246],
       [-0.65892246],
       [ 1.29928014],
       [ 1.03225252],
       [-0.74793167],
       [ 1.8333354 ],
       [ 2.27838145],
       [ 1.12126173],
       [-0.21387641],
       [-0.83694088],
       [ 0.58720647],
       [ 1.03225252],
       [-0.74793167],
       [ 1.38828935],
       [-0.74793167],
       [-0.035858  ],
       [ 2.27838145],
       [ 1.74432619],
       [ 0.23116963],
       [ 0.32017884],
       [ 0.05315121],
       [-0.65892246],
       [-0.92595009],
       [-0.035858  ],
       [ 1.38828935],
       [ 0.23116963],
       [ 0.05315121],
       [-1.10396851],
       [-0.39189483],
       [-0.92595009],
       [ 0.40918805],
       [-0

In [71]:
scaler.inverse_transform([[-0.39189483]])

array([[31.00000003]])

In [72]:
#Robust scaler
X_train['Age'].quantile(0.5)

33.0

In [73]:
median = X_train['Age'].median()

In [74]:
q1 = X_train['Age'].quantile(0.25)

In [75]:
q3 = X_train['Age'].quantile(0.75)

## Transformar las variables categóricas

In [76]:
from sklearn.preprocessing import OneHotEncoder

In [82]:
encoder = OneHotEncoder(sparse=False, drop ='first')
encoder.fit_transform(train[["Sex"]])

array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],

In [80]:
encoder.categories_

[array(['female', 'male'], dtype=object)]

In [81]:
encoder = OneHotEncoder(sparse=True)
encoder.fit_transform(train[["Sex"]])

<700x2 sparse matrix of type '<class 'numpy.float64'>'
	with 700 stored elements in Compressed Sparse Row format>

In [85]:
encoder.inverse_transform([[0]])

array([['female']], dtype=object)

## Pipeline: un objeto que almacena transformaciones que hay que aplicarle a los datos

In [87]:
from sklearn.pipeline import Pipeline

In [92]:
numeric_pipeline = Pipeline(
    [
        ('Imputacion con la media', SimpleImputer(strategy = 'mean')),
        ('Escaladominmax',MinMaxScaler())
        
    ]
)

In [94]:
numeric_pipeline.fit(train[['Age','Credit amount', 'Duration']])

Pipeline(steps=[('Imputacion con la media', SimpleImputer()),
                ('Escaladominmax', MinMaxScaler())])

In [97]:
numeric_pipeline[0].statistics_

array([  35.40285714, 3236.12142857,   21.04857143])

In [99]:
numeric_pipeline[1].min_

array([-0.36363636, -0.01375592, -0.05882353])

In [100]:
numeric_pipeline[1].max_

AttributeError: 'MinMaxScaler' object has no attribute 'max_'